In [1]:
import sys
import torch
import torchvision
from torchvision.utils import make_grid
from torch.utils.data import dataloader
sys.path.append('models/detection')
sys.path.append('models/detection/FasterRCNN')
sys.path.append('utils')

In [2]:
from models.detection.models import fasterrcnn_mobilenet_v3_large_320_fpn_COCO, fasterrcnn_resnet50_fpn_COCO
from utils.evaluate_utils import simple_watcher, AdvDetectionMetrics
from utils.visualize_utils import show, draw_bbox_with_tensor
from utils.utils import PatternProjector, ToxicTargetsGenerator, log, LossManager

from adv_patch_generator import NoiseLikePatch
from data import coco_2017_dev_5k
from trainer import AdvPatchTrainer

In [3]:
train_ds, valid_ds, _ = coco_2017_dev_5k(split_rate=[200, 1000, 5000 - 1200])
# net2attack = fasterrcnn_mobilenet_v3_large_320_fpn_COCO()
net2attack = fasterrcnn_resnet50_fpn_COCO()
patch_generator = NoiseLikePatch(128, 128)
projector = PatternProjector(mix_rate=0.99, min_luminance=-2.01, luminance_smooth_boundary=0.001)

loading annotations into memory...
Done (t=0.51s)
creating index...
index created!


In [4]:
loss_weight = {
    'atk_loss_classifier': 0.0,
    'atk_loss_box_reg': 0.0,
    'atk_loss_objectness': 1.0,
    'atk_loss_rpn_box_reg': 0.0,
    'loss_classifier': - 0.0,
    'loss_box_reg': - 0.0,
    'loss_objectness': - 1.0,
    'loss_rpn_box_reg': - 0.0,
}

In [5]:
loss_func = LossManager(**loss_weight)
optimizer = torch.optim.Adam(patch_generator.parameters(), lr=0.1)
targets_generator = ToxicTargetsGenerator()
trainer = AdvPatchTrainer(net2attack,
                          patch_generator,
                          projector,
                          loss_func,
                          optimizer,
                          targets_generator=targets_generator)

In [6]:
# watcher:...........
# train..............

In [7]:
patch = torch.load('Exps/01-01-12-34/Data/patch.pt')

In [8]:
metric = AdvDetectionMetrics(net2attack, projector, patch, None)

In [9]:
metric.compute(train_ds, True, batch_size=2)

Processing: 100%|██████████| 200/200 [00:41<00:00,  5.79it/s]

Computing metrics. It (especially 'mAP' items) may take few minutes.


Processing: 100%|██████████| 200/200 [03:59<00:00,  1.20s/it]


{'Statistic_Info': {'num_boxes_clean_images': 6829,
  'num_boxes_clean_images_outside_patch': 6556,
  'num_boxes_with_patch': 16330,
  'num_boxes_unsuppressed': 3722},
 'Average_Boxes_Number_Increase': 63.04,
 'Boxes_Suppression_Rate': 0.4322757779133618,
 'mAPs_clean_image': {'map': tensor(0.4090),
  'map_50': tensor(0.6388),
  'map_75': tensor(0.4185),
  'map_small': tensor(0.1905),
  'map_medium': tensor(0.3404),
  'map_large': tensor(0.5652),
  'mar_1': tensor(0.3403),
  'mar_10': tensor(0.4969),
  'mar_100': tensor(0.5115),
  'mar_small': tensor(0.2631),
  'mar_medium': tensor(0.4500),
  'mar_large': tensor(0.6488),
  'map_per_class': tensor(-1.),
  'mar_100_per_class': tensor(-1.)},
 'mAPs_with_patch': {'map': tensor(0.2295),
  'map_50': tensor(0.3912),
  'map_75': tensor(0.2323),
  'map_small': tensor(0.1434),
  'map_medium': tensor(0.2171),
  'map_large': tensor(0.2879),
  'mar_1': tensor(0.1896),
  'mar_10': tensor(0.2809),
  'mar_100': tensor(0.2918),
  'mar_small': tensor(0.

In [12]:
import pprint as pp
pp.pprint(metric.metrics)

{'Average_Boxes_Number_Increase': 63.04,
 'Boxes_Suppression_Rate': 0.4322757779133618,
 'Statistic_Info': {'num_boxes_clean_images': 6829,
                    'num_boxes_clean_images_outside_patch': 6556,
                    'num_boxes_unsuppressed': 3722,
                    'num_boxes_with_patch': 16330},
 'mAPs_clean_image': {'map': tensor(0.4090),
                      'map_50': tensor(0.6388),
                      'map_75': tensor(0.4185),
                      'map_large': tensor(0.5652),
                      'map_medium': tensor(0.3404),
                      'map_per_class': tensor(-1.),
                      'map_small': tensor(0.1905),
                      'mar_1': tensor(0.3403),
                      'mar_10': tensor(0.4969),
                      'mar_100': tensor(0.5115),
                      'mar_100_per_class': tensor(-1.),
                      'mar_large': tensor(0.6488),
                      'mar_medium': tensor(0.4500),
                      'mar_small': tenso